In [1]:
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
# Set dataset paths
train_dir = r'C:\Users\Dell\Documents\Projects\FaceExp\data\train'
test_dir = r'C:\Users\Dell\Documents\Projects\FaceExp\data\test'

In [3]:
# Get image sizes
def get_image_sizes(directory):
    size_dict = {}
    
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if not os.path.isdir(class_path):
            continue
        
        sizes = []
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                with Image.open(img_path) as img:
                    sizes.append(img.size)  # (width, height)
            except Exception as e:
                print(f"Error reading {img_path}: {e}")
        
        size_dict[class_name] = sizes
    
    return size_dict

In [4]:
# Print the image sizes
def print_image_sizes(directory):
    sizes = get_image_sizes(directory)
    print(f"Image sizes in {directory}:")
    for class_name, size_list in sizes.items():
        if size_list:
            unique_sizes = set(size_list)
            print(f"  {class_name}: {len(size_list)} images, Unique sizes: {unique_sizes}")
        else:
            print(f"  {class_name}: No images found")

In [5]:
print_image_sizes(train_dir)
print_image_sizes(test_dir)

Image sizes in C:\Users\Dell\Documents\Projects\FaceExp\data\train:
  angry: 3995 images, Unique sizes: {(224, 224)}
  disgust: 436 images, Unique sizes: {(224, 224)}
  fear: 4097 images, Unique sizes: {(224, 224)}
  happy: 7215 images, Unique sizes: {(224, 224)}
  neutral: 4965 images, Unique sizes: {(224, 224)}
  sad: 4830 images, Unique sizes: {(224, 224)}
  surprise: 3171 images, Unique sizes: {(224, 224)}
Image sizes in C:\Users\Dell\Documents\Projects\FaceExp\data\test:
  angry: 958 images, Unique sizes: {(224, 224)}
  disgust: 111 images, Unique sizes: {(224, 224)}
  fear: 1024 images, Unique sizes: {(224, 224)}
  happy: 1774 images, Unique sizes: {(224, 224)}
  neutral: 1233 images, Unique sizes: {(224, 224)}
  sad: 1247 images, Unique sizes: {(224, 224)}
  surprise: 831 images, Unique sizes: {(224, 224)}


In [8]:
#Resize the images according to ResNet50
def resize_images(directory, size=224):
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if not os.path.isdir(class_path):
            continue
        
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                with Image.open(img_path) as img:
                    img = img.resize((size, size), Image.LANCZOS)
                    img.save(img_path)
            except Exception as e:
                print(f"Error resizing {img_path}: {e}")

In [9]:
resize_images(train_dir)
resize_images(test_dir)